In [7]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder

In [119]:
file_path = 'C:\\hudsondata\\Machine Learning\\Redhat'
acttest = pd.read_csv(os.path.join(file_path,'act_test.csv'))
acttrain = pd.read_csv(os.path.join(file_path,'act_train.csv'))
people = pd.read_csv(os.path.join(file_path,'people.csv'))

In [3]:
print 'Acttrain has', acttrain.shape, 'acttest has', acttest.shape, 'people has',people.shape

Acttrain has (2197291, 15) acttest has (498687, 14) people has (189118, 41)


In [4]:
acttrain.head()

,people_id,activity_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,outcome
0,ppl_100,act2_1734928,2023-08-26,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 76,0
1,ppl_100,act2_2434093,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
2,ppl_100,act2_3404049,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
3,ppl_100,act2_3651215,2023-08-04,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
4,ppl_100,act2_4109017,2023-08-26,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0


In [5]:
people.head()

,people_id,char_1,group_1,char_2,date,char_3,char_4,char_5,char_6,char_7,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,ppl_100,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,...,False,True,True,False,False,True,True,True,False,36
1,ppl_100002,type 2,group 8688,type 3,2021-01-06,type 28,type 9,type 5,type 3,type 11,...,False,True,True,True,True,True,True,True,False,76
2,ppl_100003,type 2,group 33592,type 3,2022-06-10,type 4,type 8,type 5,type 2,type 5,...,False,False,True,True,True,True,False,True,True,99
3,ppl_100004,type 2,group 22593,type 3,2022-07-20,type 40,type 25,type 9,type 4,type 16,...,True,True,True,True,True,True,True,True,True,76
4,ppl_100006,type 2,group 6534,type 3,2022-07-27,type 40,type 25,type 9,type 3,type 8,...,False,False,True,False,False,False,True,True,False,84


# Pre-Process People File

In [ ]:
def pplfile():
    TFchar = list(people.columns[11:39])
    for element in TFchar:
        le = LabelEncoder()
        le.fit(people['char_11'])
        people[element] = le.transform(people[element])
    for col in people.iloc[:,[1,2,3,5,6,7,8,9,10,11]]:
        people[col] = [d.split()[1] for d in people[col]]
    people['month'] = [d.split('-')[1] for d in people.date]
    people['day'] = [d.split('-')[2] for d in people.date]
    people['year']= [d.split('-')[0] for d in people.date]
    people.drop('date',1, inplace=True)
    return people

# Pre-Process Activity File

In [121]:
def actfile(actfile):
    frames = []
    for col in actfile.iloc[:,4:14]:
        actchar = actfile[col].str.split(expand=True)[1]
        frames.append(actchar)
    actchar = pd.concat(frames,axis=1).fillna(0)
    actchar.columns = ['achar1','achar2','achar3','achar4','achar5','achar6','achar7','achar8','achar9','achar10']
    actfile['activity_category'] = [d.split()[1] for d in actfile.activity_category]
    actfile['amonth'] = [d.split('-')[1] for d in actfile.date]
    actfile['aday'] = [d.split('-')[2] for d in actfile.date]
    actfile['ayear']= [d.split('-')[0] for d in actfile.date]
    actfile.drop(['date','char_1','char_2','char_3','char_4','char_5','char_6','char_7','char_8','char_9','char_10'],1, inplace=True)
    actfile = actfile.join(actchar)
    return actfile

# Merge People File and Activity File

In [133]:
def merge(people, actfile):
    merged = pd.merge(actfile, people, on='people_id', how='left')
    merged.drop('people_id',1, inplace=True)
    merged = merged.set_index('activity_id')
    y = 'outcome'
    if y in list(merged.columns):
        y = merged['outcome']
        y=y.astype('category')
        merged.drop('outcome',1,inplace=True)        
        return merged, y
    else: 
        return merged,y

In [115]:
train, y = merge(people,acttrain)

In [120]:
people = pplfile()
people.head()

,people_id,char_1,group_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,...,char_32,char_33,char_34,char_35,char_36,char_37,char_38,month,day,year
0,ppl_100,2,17304,2,5,5,5,3,11,2,...,0,0,1,1,1,0,36,06,29,2021
1,ppl_100002,2,8688,3,28,9,5,3,11,2,...,1,1,1,1,1,0,76,01,06,2021
2,ppl_100003,2,33592,3,4,8,5,2,5,2,...,1,1,1,0,1,1,99,06,10,2022
3,ppl_100004,2,22593,3,40,25,9,4,16,2,...,1,1,1,1,1,1,76,07,20,2022
4,ppl_100006,2,6534,3,40,25,9,3,8,2,...,0,0,0,1,1,0,84,07,27,2022


In [122]:
acttrain = actfile(acttrain)
acttrain.head()

,people_id,activity_id,activity_category,outcome,amonth,aday,ayear,achar1,achar2,achar3,achar4,achar5,achar6,achar7,achar8,achar9,achar10
0,ppl_100,act2_1734928,4,0,08,26,2023,0,0,0,0,0,0,0,0,0,76
1,ppl_100,act2_2434093,2,0,09,27,2022,0,0,0,0,0,0,0,0,0,1
2,ppl_100,act2_3404049,2,0,09,27,2022,0,0,0,0,0,0,0,0,0,1
3,ppl_100,act2_3651215,2,0,08,04,2023,0,0,0,0,0,0,0,0,0,1
4,ppl_100,act2_4109017,2,0,08,26,2023,0,0,0,0,0,0,0,0,0,1


In [123]:
acttest = actfile(acttest)
acttest.head()

,people_id,activity_id,activity_category,amonth,aday,ayear,achar1,achar2,achar3,achar4,achar5,achar6,achar7,achar8,achar9,achar10
0,ppl_100004,act1_249281,1,07,20,2022,5,10,5,1,6,1,1,7,4,0
1,ppl_100004,act2_230855,5,07,20,2022,0,0,0,0,0,0,0,0,0,682
2,ppl_10001,act1_240724,1,10,14,2022,12,1,5,4,6,1,1,13,10,0
3,ppl_10001,act1_83552,1,11,27,2022,20,10,5,4,6,1,1,5,5,0
4,ppl_10001,act2_1043301,5,10,15,2022,0,0,0,0,0,0,0,0,0,3015


In [134]:
train,y = merge(people,acttrain)

In [135]:
train.head()

,activity_category,amonth,aday,ayear,achar1,achar2,achar3,achar4,achar5,achar6,...,char_32,char_33,char_34,char_35,char_36,char_37,char_38,month,day,year
activity_id,,,,,,,,,,,,,,,,,,,,,
act2_1734928,4,08,26,2023,0,0,0,0,0,0,...,0,0,1,1,1,0,36,06,29,2021
act2_2434093,2,09,27,2022,0,0,0,0,0,0,...,0,0,1,1,1,0,36,06,29,2021
act2_3404049,2,09,27,2022,0,0,0,0,0,0,...,0,0,1,1,1,0,36,06,29,2021
act2_3651215,2,08,04,2023,0,0,0,0,0,0,...,0,0,1,1,1,0,36,06,29,2021
act2_4109017,2,08,26,2023,0,0,0,0,0,0,...,0,0,1,1,1,0,36,06,29,2021


In [132]:
test,y = merge(people,acttest)
test.head()

,activity_category,amonth,aday,ayear,achar1,achar2,achar3,achar4,achar5,achar6,...,char_32,char_33,char_34,char_35,char_36,char_37,char_38,month,day,year
activity_id,,,,,,,,,,,,,,,,,,,,,
act1_249281,1,07,20,2022,5,10,5,1,6,1,...,1,1,1,1,1,1,76,07,20,2022
act2_230855,5,07,20,2022,0,0,0,0,0,0,...,1,1,1,1,1,1,76,07,20,2022
act1_240724,1,10,14,2022,12,1,5,4,6,1,...,1,1,1,1,1,1,90,10,14,2022
act1_83552,1,11,27,2022,20,10,5,4,6,1,...,1,1,1,1,1,1,90,10,14,2022
act2_1043301,5,10,15,2022,0,0,0,0,0,0,...,1,1,1,1,1,1,90,10,14,2022


In [140]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import cross_val_score

In [142]:
logreg = LogisticRegression()

In [143]:
score = cross_val_score(logreg,train,y, scoring = 'roc_auc', cv=5)

In [145]:
print score.mean()

0.893226481114
